In [2]:
from ortools.sat.python import cp_model
from time import time
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import ortools
import math

#import local module
from utils import get_holiday, get_morn_con_day, add_soft_sequence_constraint, add_soft_sum_constraint, negated_bounded_span

In [11]:
# ============================= Input ============================= #

# List of staffs
staff_names = [ "อ.บริบูรณ์","อ.ณัฐฐิกานต์","อ.ปริญญา","อ.ภาวิตา",
                "อ.ธีรพล","อ.บวร","อ.ชานนท์","อ.บุญฤทธิ์","อ.กอสิน",
                "อ.พิมพ์พรรณ","อ.กรองกาญจน์"]
num_staffs = len(staff_names)

minimal_total_services = [9, 7, 5, 9, 9, 0, 9, 0, 7, 7, 9]

# Select month and year to schedule & 
month = 3
year = 2023
num_days = (datetime(year, month+1, 1) - datetime(year, month, 1)).days
num_weeks = math.ceil(num_days / 7)
date_list = [datetime(year, month, day) for day in range(1, num_days+1)]

#Get holiday list
holiday = get_holiday(year, month)

# get morning conference day
morn_con_day = get_morn_con_day(year, month)

# List of shifts
shift_names = [ 'service1', 'service1+', 'service2', 'service2+', 
                'Observe', 'Morn Con', 'EMS',  'AMD', 'Aviation doctor', 
                'off_morning', 'off_afternoon', 'off_allday']
num_shifts = len(shift_names) # 9 shifts per day, 3 off shifts

max_shifts = [  3, 3, 3, 3, 
                3, 3, 8, 10, 5, 
                100, 100, 100]

workload_weight = [12, 10, 12, 10, 10, 8, 10, 8, 10, 0, 0, 0]
shifthours = [4, 4, 4, 4, 4, 0, 7, 7, 7, 0, 0, 0]

# workload_ratio_per_staff = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1] - Not yet implement


# ============================= Create model ============================= #

model = cp_model.CpModel()

# Create shift variables.
# shifts[(s, d, t)]: staff 's' works shift 't' on day 'd'.
shifts = {}
for s in range(num_staffs):
    for d in range(num_days):
        for t in range(num_shifts):
            shifts[(s, d, t)] = model.NewBoolVar('shift_s%id%it%i' % (s, d, t))

# ============================= Required Constraints ============================= #


# For (service1 & service2 & service1+ & service2+ & observe) Each shift is assigned to exactly one staff in that day, 
for d in range(num_days):
    if holiday[d] == False:
        for t in [0, 1, 2, 3, 4]:
            model.AddExactlyOne([shifts[(s, d, t)] for s in range(num_staffs)])
    if holiday[d] == True:
        for t in [0, 1, 2, 3, 4]:
            model.Add(sum([shifts[(s, d, t)] for s in range(num_staffs)])==0)


# For morning conference day, must have 1 staff assigned to Morning Conference
for d in range(num_days):
    if morn_con_day[d] == True:
        model.AddExactlyOne([shifts[(s, d, 5)] for s in range(num_staffs)])
    if morn_con_day[d] == False:
        model.Add(sum([shifts[(s, d, 5)] for s in range(num_staffs)])==0)

# For (EMS & AMD & Air doctor) Each shift is assigned to exactly one staff in that day, include holiday
# For Air doctor, only assign to 1 staff in 1 day after 16th day
for d in range(num_days):
    for t in [6, 7, 8]:
        model.AddExactlyOne([shifts[(s, d, t)] for s in range(num_staffs)])


# Service1 & Service1+ & Service2 & Service2+ & Observe & EMS & AMD & Air doctor can't overlap
# Service1 & Service1+ & EMS & Observe & AMD & Air doctor & Morn Con can't overlap
# Assign to off shift if no shift is assigned
for s in range(num_staffs):
    for d in range(num_days):
        model.AddAtMostOne([shifts[(s, d, t)] for t in [0, 1, 2, 3, 4, 6, 7, 8]])
        model.AddAtMostOne([shifts[(s, d, t)] for t in [0, 1, 4, 5, 6, 7, 8]])


# assign to off_morning shift if Service1 & Service1+ & Observe & Morn Con & EMS & AMD is not assigned
# assign to off_afternoon shift if Service2 & Service2+ & Observe & EMS & AMD is not assigned
# assign to off_allday shift if no shift is assigned
for s in range(num_staffs):
    for d in range(num_days):
        model.Add(shifts[(s, d, 9)] + sum([shifts[(s, d, t)] for t in [0, 1, 4, 5, 6, 7, 8]]) == 1)
        model.Add(shifts[(s, d, 10)] + sum([shifts[(s, d, t)] for t in [2, 3, 4, 6, 7, 8]]) == 1)
        model.Add(shifts[(s, d, 11)] + sum([shifts[(s, d, t)] for t in [0, 1, 2, 3, 4, 5, 6, 7, 8]]) == 1)


# Fixed shift
# (staff, day, shift)
fixed_shift = [
    # AI = 1,8,9 can't work on wednesday allday and friday morning

    # (0, 0, 0), # service1
    # (0, 1, 0), # service1
    # (0, 2, 0), # service1
]

# AI instructor can't work on wednesday allday and friday morning
for s in [1,8,9]:
    for d in range(num_days):
        if date_list[d].weekday() == 2: # wednesday allday
            fixed_shift.append((s, d, 11))
        if date_list[d].weekday() == 4: # friday morning
            fixed_shift.append((s, d, 9))

for s, d, t in fixed_shift:
    model.Add(shifts[(s, d, t)] == 1)

# Fixed non-shift
# (staff, day, shift)
fixed_non_shift = [
    # (0, 0, 1), # service1+
    # (0, 1, 1), # service1+
    # (0, 2, 1), # service1+
]

for s, d, t in fixed_non_shift:
    model.Add(shifts[(s, d, t)] == 0)

# Minimal total service1, service1+, service2, service2+ for each staff
for s in range(num_staffs):
    model.Add(sum([shifts[(s, d, t)] for d in range(num_days) for t in [0, 1, 2, 3]]) >= minimal_total_services[s])


# Maximum total shift for each staff each shift
for s in range(num_staffs):
    for t in range(num_shifts):
        model.Add(sum([shifts[(s, d, t)] for d in range(num_days)]) <= max_shifts[t])

# ============================= Objectives ============================= #
obj_int_vars = []
obj_int_coeffs = []
obj_bool_vars = []
obj_bool_coeffs = []


# Regular shift
# (staff, day, shift, penalty)
requests = [
    (0, 0, 0, 1), # service1
]

for s, d, t, p in requests:
    obj_bool_vars.append(shifts[(s, d, t)])
    obj_bool_coeffs.append(p)


# Consecutive aviation doctor shift (2 days)
# (shift, hard_min, soft_min, min_penalty, hard_max, soft_max, max_penalty)
consecutive_shifts = [
    (8, 1, 2, 1000, 2, 2, 0)
]

for ct in consecutive_shifts:
        t, hard_min, soft_min, min_cost, soft_max, hard_max, max_cost = ct
        for s in range(num_staffs):
            works = [shifts[s, d, t] for d in range(num_days)]
            variables, coeffs = add_soft_sequence_constraint(
                model, works, hard_min, soft_min, min_cost, soft_max, hard_max,
                max_cost,
                f'consecutive_shifts(staff {staff_names[p]}, {shift_names[t]})')
            obj_bool_vars.extend(variables)
            obj_bool_coeffs.extend(coeffs)


# Minimal number of different staffs each week for EMS, Observe
days_in_week = [5, 7, 7, 7, 5]
for w in range(num_weeks):
    for t in [6, 7]:
        works = []
        for s in range(num_staffs):
            work = model.NewIntVar(0, 1, f'Week {w+1} {shift_names[t]} {staff_names[s]} work')
            model.AddMaxEquality(work,[shifts[(s, d, t)] for d in range(sum(days_in_week[:w]),sum(days_in_week[:w+1]))])
            works.append(work)
        # print(sum(works))
        # num_staffs_each_week = model.NewIntVar(0, num_staffs, f'Week {w+1} {shift_names[t]} num_staffs')
        obj_int_vars.append(sum(works))
        obj_int_coeffs.append(10)


#equalize workloads
workload_list = []
for s in range(num_staffs):
    workload_per_staff = model.NewIntVar(0, 1000, f'Workload of {staff_names[s]}')
    model.Add(sum([cp_model.LinearExpr.WeightedSum([shifts[(s, d, t)] for t in range(num_shifts)], workload_weight) for d in range(num_days)]) == workload_per_staff)
    workload_list.append(workload_per_staff)
    for s2 in range(num_staffs):
        if s2 == s:
            continue
        workload_per_staff2 = model.NewIntVar(0, 1000, f'Workload of {staff_names[s2]}')
        model.Add(sum([cp_model.LinearExpr.WeightedSum([shifts[(s2, d, t)] for t in range(num_shifts)], workload_weight) for d in range(num_days)]) == workload_per_staff2)
        delta = model.NewIntVar(0, 1000, f'Delta of {staff_names[s]} and {staff_names[s2]}')
        model.AddAbsEquality(delta, workload_per_staff - workload_per_staff2)
        obj_int_vars.append(delta)
        obj_int_coeffs.append(1)


obj_bool_penalties = sum([coeff * variables for coeff, variables in zip(obj_bool_coeffs, obj_bool_vars)])
obj_int_penalties = sum([coeff * variables for coeff, variables in zip(obj_int_coeffs, obj_int_vars)])
penalties = obj_bool_penalties + obj_int_penalties
model.Minimize(penalties)


# ============================= Solve ============================= #


# Solution printer.
class ShiftSolutionPrinter(cp_model.CpSolverSolutionCallback):

    def __init__(self, shifts, num_staffs, num_days, num_shifts, workload_list, penalties):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._shifts = shifts
        self._num_staffs = num_staffs
        self._num_days = num_days
        self._num_shifts = num_shifts
        self._solution_count = 0
        self._solutions = []
        self._workloads = workload_list
        self._penalties = penalties

    def OnSolutionCallback(self):
        self._solution_count += 1

        print(f'Solution {self._solution_count}')
        print(f' Penalty = {self.Value(self._penalties)}')

        # Count shift per staff
        shift_count = np.zeros((self._num_staffs, self._num_shifts))
        for s in range(self._num_staffs):
            for d in range(self._num_days):
                for t in range(self._num_shifts):
                    shift_count[s, t] += self.Value(self._shifts[(s, d, t)])
        df_shift_count = pd.DataFrame(shift_count, index=staff_names, columns=shift_names)
        #Sum service1, 1+, 2, 2+
        df_shift_count['total_services'] = df_shift_count['service1'] + df_shift_count['service1+'] + df_shift_count['service2'] + df_shift_count['service2+']
        #Calculate workhours with shifthours
        df_shift_count['workhours'] = sum([shifthours[i] * df_shift_count[shift_names[i]] for i in range(len(shift_names))])
        df_shift_count['workload'] = sum([workload_weight[i] * df_shift_count[shift_names[i]] for i in range(len(shift_names))])


        # print(df_shift_count)
        df_shift_count.to_csv('shift_count.csv', index=True, header=True)

        ls=[]
        for d in range(self._num_days):
            ls_day=[]
            for w in range(self._num_shifts):
                ls_shift=[]
                for s in range(self._num_staffs):
                    if self.Value(self._shifts[(s, d, w)]):
                        ls_shift.append(staff_names[s])
                ls_day.append(ls_shift)
            ls.append(ls_day)

        # print(self._shifts[(0, 0, 0)].Value())
        # print(self._workloads[0].Value())
        # for s in range(self._num_staffs):
        #     print(self.Value(self._workloads[s]))
        
        df = pd.DataFrame(ls, index=date_list, columns=shift_names)
        # print(df)

    

        df.to_csv('schedule.csv', index=True, header=True)
        self._solutions.append(df)

    def SolutionCount(self):
        return self._solution_count

    def get_solutions(self):
        return self._solutions

    def get_solution(self):
        return self._solutions.pop()



# Solve model.
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 600
solution_printer = ShiftSolutionPrinter(shifts, num_staffs, num_days, num_shifts, workload_list, penalties)
status = solver.Solve(model, solution_printer)
# status = solver.Solve(model)
print(solver.ResponseStats())



Solution 1
 Penalty = 20994
Solution 2
 Penalty = 11296
Solution 3
 Penalty = 8710
Solution 4
 Penalty = 8702
Solution 5
 Penalty = 8688
Solution 6
 Penalty = 8608
Solution 7
 Penalty = 8290
Solution 8
 Penalty = 8214
Solution 9
 Penalty = 8120
Solution 10
 Penalty = 8062
Solution 11
 Penalty = 8040
Solution 12
 Penalty = 7840
Solution 13
 Penalty = 7826
Solution 14
 Penalty = 7788
Solution 15
 Penalty = 7708
Solution 16
 Penalty = 7662
Solution 17
 Penalty = 7654
Solution 18
 Penalty = 6906
Solution 19
 Penalty = 6802
Solution 20
 Penalty = 6716
Solution 21
 Penalty = 6702
Solution 22
 Penalty = 6678
Solution 23
 Penalty = 6668
Solution 24
 Penalty = 6366
Solution 25
 Penalty = 6288
Solution 26
 Penalty = 6280
Solution 27
 Penalty = 6084
Solution 28
 Penalty = 6064
Solution 29
 Penalty = 5986
Solution 30
 Penalty = 5970
Solution 31
 Penalty = 5962
Solution 32
 Penalty = 5808
Solution 33
 Penalty = 2952
Solution 34
 Penalty = 2942
Solution 35
 Penalty = 2932
Solution 36
 Penalty = 2922

In [5]:
#equalize workloads
for s in range(num_staffs):
    workload_per_staff = model.NewIntVar(0, 1000, f'Workload of {staff_names[s]}')
    model.Add(sum([cp_model.LinearExpr.WeightedSum([shifts[(s, d, t)] for t in range(num_shifts)], workload_weight) for d in range(num_days)]) == workload_per_staff)

    for s2 in range(num_staffs):
        if s2 == s:
            continue
        workload_per_staff2 = model.NewIntVar(0, 1000, f'Workload of {staff_names[s2]}')
        model.Add(sum([cp_model.LinearExpr.WeightedSum([shifts[(s2, d, t)] for t in range(num_shifts)], workload_weight) for d in range(num_days)]) == workload_per_staff2)
        delta = model.NewIntVar(0, 1000, f'Delta of {staff_names[s]} and {staff_names[s2]}')
        model.Add(delta == workload_per_staff - workload_per_staff2)
        obj_int_vars.append(delta)
        obj_int_coeffs.append(100)



In [74]:
#equalize workloads
workload_list = []
for s in range(num_staffs):
    for d in range(num_days):
        workload_per_day = model.NewIntVar(0, 100, f'Workload of {staff_names[s]} on {date_list[d]}')
        model.Add(cp_model.LinearExpr.WeightedSum([shifts[(s, d, t)] for t in range(num_shifts)], workload_weight) == workload_per_day)
    workload = model.NewIntVar(0, 1000, f'Workload of {staff_names[s]}')
    model.Add(workload == sum([workload_per_day for d in range(num_days)]))
    workload_list.append(workload)

# average_workload = model.NewIntVar(0, 1000, f'Average workload')
# model.AddDivisionEquality(average_workload, sum([workload for s in range(num_staffs)]), num_staffs)

# delta = model.NewIntVar(0, 100, "")
# for s in range(num_staffs):
#     model.Add(delta >= workload_list[s] - average_workload)
#     model.Add(delta >= average_workload - workload_list[s])

# obj_int_vars.append(delta)
# obj_int_coeffs.append(1)

print(workload_list)



    # model.Add(sum([shifts[(s, d, t)] * workload_weight[t] for d in range(num_days) for t in range(num_shifts-3)]) == workload)



[Workload of อ.บริบูรณ์(0..1000), Workload of อ.ณัฐฐิกานต์(0..1000), Workload of อ.ปริญญา(0..1000), Workload of อ.ภาวิตา(0..1000), Workload of อ.ธีรพล(0..1000), Workload of อ.บวร(0..1000), Workload of อ.ชานนท์(0..1000), Workload of อ.บุญฤทธิ์(0..1000), Workload of อ.กอสิน(0..1000), Workload of อ.พิมพ์พรรณ(0..1000), Workload of อ.กรองกาญจน์(0..1000)]


In [133]:
obj_int_vars = []
obj_int_coeffs = []

# Minimal number of different staffs each week for EMS, Observe & cut week on sunday

# Minimal number of different staffs each week for EMS, Observe
days_in_week = [5, 7, 7, 7, 5]
for w in range(num_weeks):
    # print(range(sum(days_in_week[:w]),sum(days_in_week[:w+1])))
    # print(sum(days_in_week[w]))
    for t in [6, 7]:
        works = []
        for s in range(num_staffs):
            work = model.NewIntVar(0, 1, f'Week {w+1} {shift_names[t]} {staff_names[s]} work')
            model.AddMaxEquality(work,[shifts[(s, d, t)] for d in range(sum(days_in_week[:w]),sum(days_in_week[:w+1]))])
            # works.append(work)
        # print(sum(works))
        # num_staffs_each_week = model.NewIntVar(0, num_staffs, f'Week {w+1} {shift_names[t]} num_staffs')
        # obj_int_vars.append(sum(works))
        # obj_int_coeffs.append(1)


range(0, 5)
range(5, 12)
range(12, 19)
range(19, 26)
range(26, 31)


In [84]:
works = []
for t in [6, 7]:
    for d in date_list:
        for s in range(num_staffs):
            # work = model.NewIntVar(0, 1, f'Day {d.date()} {shift_names[t]} {staff_names[s]} work')
            work = shifts[(s, d.day-1, t)]
            works.append(work)
        
        if (d.weekday() == 6) | (d == date_list[-1]):
            num_staffs_each_week = model.NewIntVar(0, num_staffs, f'Week {d.day//7 +1} ({len(works) // num_staffs} days) {shift_names[t]} num_staffs')
            # print(num_staffs_each_week)
            # print(sum(works))
            model.Add(num_staffs_each_week == sum(works))
            obj_int_vars.append(num_staffs_each_week)
            obj_int_coeffs.append(1)

            print(sum(works))
            works = []
            continue
    
    



((((((((((((((((((((((((((((((((((((((((((((((((((((((shift_s0d0t6 + shift_s1d0t6) + shift_s2d0t6) + shift_s3d0t6) + shift_s4d0t6) + shift_s5d0t6) + shift_s6d0t6) + shift_s7d0t6) + shift_s8d0t6) + shift_s9d0t6) + shift_s10d0t6) + shift_s0d1t6) + shift_s1d1t6) + shift_s2d1t6) + shift_s3d1t6) + shift_s4d1t6) + shift_s5d1t6) + shift_s6d1t6) + shift_s7d1t6) + shift_s8d1t6) + shift_s9d1t6) + shift_s10d1t6) + shift_s0d2t6) + shift_s1d2t6) + shift_s2d2t6) + shift_s3d2t6) + shift_s4d2t6) + shift_s5d2t6) + shift_s6d2t6) + shift_s7d2t6) + shift_s8d2t6) + shift_s9d2t6) + shift_s10d2t6) + shift_s0d3t6) + shift_s1d3t6) + shift_s2d3t6) + shift_s3d3t6) + shift_s4d3t6) + shift_s5d3t6) + shift_s6d3t6) + shift_s7d3t6) + shift_s8d3t6) + shift_s9d3t6) + shift_s10d3t6) + shift_s0d4t6) + shift_s1d4t6) + shift_s2d4t6) + shift_s3d4t6) + shift_s4d4t6) + shift_s5d4t6) + shift_s6d4t6) + shift_s7d4t6) + shift_s8d4t6) + shift_s9d4t6) + shift_s10d4t6)
((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((

In [125]:
# assign to off_morning shift if Service1 & Service1+ & Observe & Morn Con & EMS & AMD is not assigned
# assign to off_afternoon shift if Service2 & Service2+ & Observe & EMS & AMD is not assigned
for s in range(num_staffs):
    for d in range(num_days):
        model.Add(sum([shifts[(s, d, t)] for t in [1, 3, 5, 6, 7, 8]])==0).OnlyEnforceIf(shifts[(s, d, 9)])
        model.Add(sum([shifts[(s, d, t)] for t in [2, 4, 5, 7, 8]])==0).OnlyEnforceIf(shifts[(s, d, 10)])
            # model.Add(shifts[(s, d, 9)] == 1).OnlyEnforceIf(shifts[(s, d, t)])



In [51]:
# ============================= Output ============================= #

solution_printer.get_solution()

,off,service1,service2,service1+,service2+,Observe,Morn Con,EMS,AMD
2023-03-01,"[อ.บวร, อ.ชานนท์, อ.บุญฤทธิ์, อ.กอสิน]",[อ.ภาวิตา],[อ.บริบูรณ์],[อ.ปริญญา],[อ.ณัฐฐิกานต์],[อ.ธีรพล],[],[อ.พิมพ์พรรณ],[อ.กรองกาญจน์]
2023-03-02,"[อ.บวร, อ.ชานนท์, อ.บุญฤทธิ์, อ.กอสิน]",[อ.ธีรพล],[อ.บริบูรณ์],[อ.พิมพ์พรรณ],[อ.ภาวิตา],[อ.ปริญญา],[],[อ.ณัฐฐิกานต์],[อ.กรองกาญจน์]
2023-03-03,"[อ.ชานนท์, อ.บุญฤทธิ์, อ.กอสิน, อ.พิมพ์พรรณ]",[อ.ธีรพล],[อ.ภาวิตา],[อ.ณัฐฐิกานต์],[อ.ปริญญา],[อ.กรองกาญจน์],[],[อ.บวร],[อ.บริบูรณ์]
2023-03-04,"[อ.บริบูรณ์, อ.ณัฐฐิกานต์, อ.ภาวิตา, อ.ธีรพล, ...",[],[],[],[],[],[],[อ.ปริญญา],[อ.กรองกาญจน์]
2023-03-05,"[อ.บริบูรณ์, อ.ณัฐฐิกานต์, อ.ปริญญา, อ.ภาวิตา,...",[],[],[],[],[],[],[อ.พิมพ์พรรณ],[อ.กรองกาญจน์]
2023-03-06,"[อ.ณัฐฐิกานต์, อ.ปริญญา, อ.ภาวิตา, อ.ธีรพล, อ....",[],[],[],[],[],[],[อ.กรองกาญจน์],[อ.บริบูรณ์]
2023-03-07,"[อ.ภาวิตา, อ.ชานนท์, อ.บุญฤทธิ์, อ.กอสิน]",[อ.พิมพ์พรรณ],[อ.ณัฐฐิกานต์],[อ.กรองกาญจน์],[อ.ธีรพล],[อ.ปริญญา],[],[อ.บริบูรณ์],[อ.บวร]
2023-03-08,"[อ.ภาวิตา, อ.ชานนท์, อ.บุญฤทธิ์, อ.กอสิน]",[อ.พิมพ์พรรณ],[อ.ณัฐฐิกานต์],[อ.กรองกาญจน์],[อ.ธีรพล],[อ.บวร],[],[อ.ปริญญา],[อ.บริบูรณ์]
2023-03-09,"[อ.ภาวิตา, อ.ชานนท์, อ.บุญฤทธิ์, อ.กอสิน]",[อ.ธีรพล],[อ.กรองกาญจน์],[อ.พิมพ์พรรณ],[อ.บวร],[อ.ณัฐฐิกานต์],[],[อ.บริบูรณ์],[อ.ปริญญา]
2023-03-10,"[อ.ภาวิตา, อ.ธีรพล, อ.ชานนท์, อ.บุญฤทธิ์]",[อ.บวร],[อ.พิมพ์พรรณ],[อ.บริบูรณ์],[อ.ปริญญา],[อ.กรองกาญจน์],[],[อ.กอสิน],[อ.ณัฐฐิกานต์]
